```
<>, empty garbage.
<random characters>, garbage containing random characters.
<<<<>, because the extra < are ignored.
<{!>}>, because the first > is canceled.
<!!>, because the second ! is canceled, allowing the > to terminate the garbage.
<!!!>>, because the second ! and the first > are canceled.
<{o"i!a,<{i<a>, which ends at the first >.

{}, 1 group.
{{{}}}, 3 groups.
{{},{}}, also 3 groups.
{{{},{},{{}}}}, 6 groups.
{<{},{},{{}}>}, 1 group (which itself contains garbage).
{<a>,<a>,<a>,<a>}, 1 group.
{{<a>},{<a>},{<a>},{<a>}}, 5 groups.
{{<!>},{<!>},{<!>},{<a>}}, 2 groups (since all but the last > are canceled).
```

In [1]:
from enum import Enum

In [2]:
GROUP_SEPARATOR = ','
START_GROUP = '{'
END_GROUP = '}'
START_GARBAGE = '<'
END_GARBAGE = '>'
IGNORE = '!'

class State(Enum):
    OUTSIDE_GROUP = GROUP_SEPARATOR
    IN_GROUP = START_GROUP
    GARBAGE = START_GARBAGE
    IGNORE = IGNORE

In [3]:
def outside_group_move(character, state):
    state_transition_map = {
        START_GARBAGE: (0, [State.OUTSIDE_GROUP, State.GARBAGE], False, False),
        START_GROUP: (0, [State.IN_GROUP], False, False),
        END_GROUP: (-1, [State.OUTSIDE_GROUP], True, False)
    }
    if character in state_transition_map:
        return state_transition_map[character]
    else:
        return 0, [State.OUTSIDE_GROUP], False, False

In [4]:
def in_group_move(character, state):
    state_transition_map = {
        START_GROUP: (1, [State.IN_GROUP], False, False),
        END_GROUP: (0, [State.OUTSIDE_GROUP], True, False),
        START_GARBAGE: (0, [State.IN_GROUP,State.GARBAGE], False, False),
    }
    if character in state_transition_map:
        return state_transition_map[character]
    else:
        return 0, [State.IN_GROUP], False, False

In [5]:
def garbage_move(character, state):
    s = list(state)
    if character == END_GARBAGE:
        s.pop()
        return 0, s, False, False
    
    if character == IGNORE:
        s[-1] = State.IGNORE
        return 0, s, False, False
    
    return 0, state, False, True
   

In [6]:
assert garbage_move('>', [State.IN_GROUP, State.GARBAGE]) == (0, [State.IN_GROUP], False, False)
assert garbage_move('!', [State.IN_GROUP, State.GARBAGE]) == (0, [State.IN_GROUP, State.IGNORE], False, False)

In [7]:
def ignore_move(character, state):
    s = list(state)
    s[-1] = State.GARBAGE
    return 0, s, False, False


In [8]:
move_map = {
    State.OUTSIDE_GROUP: outside_group_move,
    State.IN_GROUP: in_group_move,
    State.GARBAGE: garbage_move,
    State.IGNORE: ignore_move,   
}

```
{}, score of 1.
{{{}}}, score of 1 + 2 + 3 = 6.
{{},{}}, score of 1 + 2 + 2 = 5.
{{{},{},{{}}}}, score of 1 + 2 + 3 + 3 + 3 + 4 = 16.
{<a>,<a>,<a>,<a>}, score of 1.
{{<ab>},{<ab>},{<ab>},{<ab>}}, score of 1 + 2 + 2 + 2 + 2 = 9.
{{<!!>},{<!!>},{<!!>},{<!!>}}, score of 1 + 2 + 2 + 2 + 2 = 9.
{{<a!>},{<a!>},{<a!>},{<ab>}}, score of 1 + 2 = 3.
```

In [9]:
def solve(stream):
    state = [State.IN_GROUP]
    level = 0
    score = 0
    garbage = 0
    for c in stream:
        level_change, state, does_score, is_garbage = move_map[state[-1]](c, state)
        #print(c, level_change, state, does_score)
        level += level_change
        if does_score:
            score += level
        if is_garbage:
            garbage += 1
    return score, garbage   

In [10]:
assert solve('{}') == (1, 0)
assert solve('{{{}}}') == (6, 0)
assert solve('{{},{}}') == (5, 0)
assert solve('{{{},{},{{}}}}') == (16, 0)
assert solve('{<a>,<a>,<a>,<a>}') == (1, 4)
assert solve('{{<ab>},{<ab>},{<ab>},{<ab>}}') == (9, 8)
assert solve('{{<!!>},{<!!>},{<!!>},{<!!>}}') == (9, 0)
assert solve('{{<a!>},{<a!>},{<a!>},{<ab>}}') == (3, 17)

In [11]:
solve('{{<a!>},{<a!>},{<a!>},{<ab>}}')


(3, 17)

In [12]:
with open('garbage_stream.txt') as fh:
    input = fh.read()

In [13]:
solve(input)

(14204, 6622)